In [1]:
pip install surprise

     |████████████████████████████████| 11.8 MB 3.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630203 sha256=9fa01933ee16a638367b431eb8b432b6ae2204da94eba34bb16506a020c22f1a
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [2]:
from surprise import Reader, Dataset, SVD
from surprise.accuracy import rmse, mae
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np

import os
from scipy import sparse
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
import random

from surprise.model_selection import train_test_split

from surprise.model_selection import KFold
from surprise import accuracy

In [3]:
#Unzip the file
!unzip book1M.zip

Archive:  book1M.zip
   creating: book1M/
  inflating: book1M/BX-Book-Ratings.csv  
  inflating: book1M/BX-Books.csv     
  inflating: book1M/BX-Users.csv     
  inflating: book1M/explicit_ratings_books.csv  


In [4]:
book = pd.read_csv('book1M/BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 

In [5]:
#reading data
user = pd.read_csv('book1M/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user.columns = ['userID', 'Location', 'Age']
ratings = pd.read_csv('book1M/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
ratings.columns = ['userID', 'ISBN', 'Ratings']
book = pd.read_csv('book1M/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
book.columns = ['userID', 'Location', 'Age']
df = pd.merge(user, ratings, on='userID', how='inner')
df.drop(['Location', 'Age'], axis=1, inplace=True)

df.head(10)

df.describe()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,userID,Ratings
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [7]:

min_book_ratings = 50
filter_books = df['ISBN'].value_counts() > min_book_ratings
filter_books = filter_books[filter_books].index.tolist()

min_user_ratings = 50
filter_users = df['userID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['ISBN'].isin(filter_books)) & (df['userID'].isin(filter_users))]
#df_new = df_new.fillna(0)
df_new.head(10)

,userID,ISBN,Ratings
394,243,0060915544,10
395,243,0060977493,7
397,243,0156006529,0
400,243,0316096199,0
401,243,0316601950,9
402,243,0316602906,0
403,243,0316666343,0
404,243,0316693006,0
405,243,0316776963,9
406,243,0316899984,7


In [8]:
print('No. of Unique Users    :', df_new.userID.nunique())
print('No. of Unique ISBN :', df_new.ISBN.nunique())
print('Unique Ratings  :', df_new.Ratings.unique())
#since the unique value of ratings is 11: 0 - 10, no NaN
ratings_per_user = df_new['Ratings'].count()/df_new.userID.nunique()
print('ratings per user',ratings_per_user)
ratings_per_book = df_new['Ratings'].count()/df_new.ISBN.nunique()
print('ratings per book',ratings_per_book)

No. of Unique Users    : 3237
No. of Unique ISBN : 2125
Unique Ratings  : [10  7  0  9  6  8  5  4  1  2  3]
ratings per user 43.4093296261971
ratings per book 66.12517647058823


In [9]:
ratings_matrix = df_new.pivot(index='userID', columns='ISBN', values='Ratings')
userID = ratings_matrix.index
bookID = ratings_matrix.columns

In [14]:
#Surprise Read
reader = Reader()

data = Dataset.load_from_df(ratings[['userID','ISBN','Ratings']],reader)
trainset, testset = train_test_split(data, train_size=0.8, test_size=0.2)

In [16]:
model= SVD(n_epochs=10)
model.fit(trainset)
test_pred = model.test(testset)
print("Hasil SVD")
accuracy.rmse(test_pred, verbose=True)

Hasil SVD
RMSE: 3.4813


3.481347360006032

In [ ]:

# Run 5-fold cross-validation and then print results
cross_validate(model, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.5789  3.5558  3.5435  3.5451  3.5448  3.5536  0.0134  
Fit time          6.68    6.65    6.65    6.63    6.64    6.65    0.02    
Test time         0.45    0.24    0.48    0.40    0.39    0.39    0.08    


{'fit_time': (6.682087182998657,
  6.646923303604126,
  6.654062509536743,
  6.633181810379028,
  6.639828205108643),
 'test_rmse': array([3.57894908, 3.55579058, 3.54347802, 3.5451085 , 3.54477659]),
 'test_time': (0.44875526428222656,
  0.23726606369018555,
  0.4806385040283203,
  0.3958280086517334,
  0.3877394199371338)}

In [ ]:
ratings.shape

(1149780, 3)

In [ ]:
n_users = user.shape[0]
n_books = book.shape[0]
print (n_users * n_books)

75670906880


In [ ]:
n_ratings=ratings[ratings.ISBN.isin(book.ISBN)]
n_ratings= n_ratings[n_ratings.userID.isin(user.userID)]

In [ ]:
sparsity = 1.0-len(n_ratings)/float(n_users*n_books)
print ('the sparsity bookcrossing = ' + str(sparsity*100) + '%')

the sparsity bookcrossing = 99.99863734155898%
